We are using the dataset lisa_tiny and yolo v5

In [10]:
# setting up yolo v5

!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -U -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.12 MiB | 18.19 MiB/s, done.
Resolving deltas: 100% (11858/11858), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [8]:
# setting up dataset

import os
import random
import shutil
import pandas as pd
from PIL import Image

# define vars
image_folder = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny'
annotation_file = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/annotations.csv'

# Create directories for train and val if they don't exist
train_images = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/train'
val_images = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/val'
train_labels = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/labels/train/'
val_labels = '/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/labels/val/'

os.makedirs(train_images, exist_ok=True)
os.makedirs(val_images, exist_ok=True)
os.makedirs(train_labels, exist_ok=True)
os.makedirs(val_labels, exist_ok=True)

# Load annotations
annotations = pd.read_csv(annotation_file)

# Get unique image filenames
image_files = annotations['filename'].unique()

# Shuffle the list of image files
random.shuffle(image_files)

# Split the dataset (80% for training, 20% for validation)
train_size = int(0.8 * len(image_files))  # 80% for training
train_files = image_files[:train_size]
val_files = image_files[train_size:]

# Move images to the appropriate directories
# for file in train_files:
#     shutil.move(os.path.join(image_folder, file), os.path.join(train_images, file))
# for file in val_files:
#     shutil.move(os.path.join(image_folder, file), os.path.join(val_images, file))

print("moved all files to respective folders")

# Map class names to class IDs
class_map = {
    'stop': 0,
    'yield': 1,
    'yieldAhead': 2,
    'merge': 3,
    'signalAhead': 4,
    'pedestrianCrossing': 5,
    'keepRight': 6,
    'speedLimit35': 7,
    'speedLimit25': 8
}

# since images are already in their respective folders
train_files = os.listdir(train_images)
val_files = os.listdir(val_images)

# Process each image in the training set
for file in train_files:
    # Get the corresponding annotations for the image
    image_annotations = annotations[annotations['filename'] == file]

    # Get image size (to normalize bounding box coordinates)
    image_path = os.path.join(train_images, file)
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Create the corresponding label file for YOLO
    label_file = os.path.join(train_labels, file.replace('.png', '.txt'))

    with open(label_file, 'w') as f:
        for _, row in image_annotations.iterrows():
            # Get bounding box coordinates
            x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
            class_name = row['class']
            class_id = class_map[class_name]  # Map class name to ID

            # Calculate YOLO format: x_center, y_center, width, height (normalized)
            x_center = (x1 + x2) / 2 / image_width
            y_center = (y1 + y2) / 2 / image_height
            width = (x2 - x1) / image_width
            height = (y2 - y1) / image_height

            # Write to label file
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process each image in the validation set
for file in val_files:
    # Get the corresponding annotations for the image
    image_annotations = annotations[annotations['filename'] == file]

    # Get image size (to normalize bounding box coordinates)
    image_path = os.path.join(val_images, file)
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Create the corresponding label file for YOLO
    label_file = os.path.join(val_labels, file.replace('.png', '.txt'))

    with open(label_file, 'w') as f:
        for _, row in image_annotations.iterrows():
            # Get bounding box coordinates
            x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
            class_name = row['class']
            class_id = class_map[class_name]  # Map class name to ID

            # Calculate YOLO format: x_center, y_center, width, height (normalized)
            x_center = (x1 + x2) / 2 / image_width
            y_center = (y1 + y2) / 2 / image_height
            width = (x2 - x1) / image_width
            height = (y2 - y1) / image_height

            # Write to label file
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("Processed and created YOLO label files for all images")

# Create the YAML file for YOLOv5
with open('/content/drive/My Drive/traffic_sign_recognition/lisa_tiny/traffic_sign.yaml', 'w') as f:
    f.write("""
    train: /content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/train
    val: /content/drive/My Drive/traffic_sign_recognition/lisa_tiny/images/val
    nc: 9  # Number of classes
    names: ['stop', 'yield', 'yieldAhead', 'merge', 'signalAhead', 'pedestrianCrossing', 'keepRight', 'speedLimit35', 'speedLimit25']
    """)


moved all files to respective folders
Processed and created YOLO label files for all images


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train the model with python
# training yolov5 with the ultralytics library.
# remember to run installation and requirements first

!python ./yolov5/train.py --img 640 --batch 16 --epochs 50 --data /content/drive/My\ Drive/traffic_sign_recognition/lisa_tiny/traffic_sign.yaml --weights yolov5s.pt --cache

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-12 00:00:54.064536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741737654.244630   10316 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741737654.293958   10316 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh